# Filtering Data

In [1]:
import json
from pathlib import Path
from csv import reader

## Load Data

In [2]:
data_folder = Path("../data/genre_playlists/").glob('**/*')
playlist_files = [x for x in data_folder if x.is_file()]

tracks_folder = Path("../data/track_features/").glob('**/*')
tracks_files = [x for x in tracks_folder if x.is_file()]

In [3]:
tracks = dict()

for f in tracks_files:
    with open(f, 'r') as infile:
        rd = reader(infile)
        row1 = next(rd)
        for line in rd:
            track_name = line[0]
            artist_name = line[1]
            tracks[track_name] = artist_name

In [4]:
playlists = []

for f in playlist_files:
    with open(f, 'r') as infile:
        data = json.load(infile)
        playlists += data

print(len(playlists))


5035


## Remove tracks

In [5]:
total_del_tracks = 0
playlists_to_del = []

#Removing tracks that aren't labeled, and playlists with <10 tracks or less than 30% of original size or >5000 tracks
for playlist in playlists:
    new_tracks = list(filter(lambda x : x['name'] in tracks, playlist['tracks']))
    orig_size = len(playlist['tracks'])
    new_size = len(new_tracks)
    if new_size < 10 or new_size < orig_size * 0.3 or new_size > 5000:
        playlists_to_del.append(playlist['name'])
        total_del_tracks += orig_size
    else:
        playlist['tracks'] = new_tracks
        playlist['num_tracks'] = new_size
        total_del_tracks += orig_size - new_size
    

In [6]:
print(total_del_tracks)
print(playlists_to_del)

5
[]


In [7]:
chunk = []
for i, pl in enumerate(playlists):
    chunk.append(pl)
    if (i + 1) % 200 == 0:
        begin = (i + 1) - 200
        with open('../data/genre_playlists_updated/playlists.{}-{}.json'.format(begin, i), 'w') as outfile:
            json.dump(chunk, outfile, indent=4)
        chunk = []
    elif i == len(playlists) - 1:
        begin = i - (i + 1) % 200
        with open('../data/genre_playlists_updated/playlists.{}-{}.json'.format(begin, i), 'w') as outfile:
            json.dump(chunk, outfile, indent=4)
        chunk = []